In [1]:
!wget https://ss-takashi.sakura.ne.jp/corpus/jvnv/jvnv_ver1.zip

--2025-12-05 18:57:34--  https://ss-takashi.sakura.ne.jp/corpus/jvnv/jvnv_ver1.zip
ss-takashi.sakura.ne.jp (ss-takashi.sakura.ne.jp) をDNSに問いあわせています... 219.94.162.51
ss-takashi.sakura.ne.jp (ss-takashi.sakura.ne.jp)|219.94.162.51|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 1778635432 (1.7G) [application/zip]
`jvnv_ver1.zip' に保存中

jvnv_ver1.zip       100%[===================>]   1.66G  1.91MB/s 時間 17m 40s    

2025-12-05 19:15:14 (1.60 MB/s) - `jvnv_ver1.zip' へ保存完了 [1778635432/1778635432]



In [2]:
!unzip jvnv_ver1.zip

Archive:  jvnv_ver1.zip
   creating: jvnv_v1
  inflating: jvnv_v1/transcription.pdf  
   creating: jvnv_v1/F1
   creating: jvnv_v1/F1/surprise
   creating: jvnv_v1/F1/surprise/free
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_08.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_10.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_06.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_04.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_02.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_05.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_09.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_03.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_01.wav  
  inflating: jvnv_v1/F1/surprise/free/F1_surprise_free_07.wav  
   creating: jvnv_v1/F1/surprise/regular
  inflating: jvnv_v1/F1/surprise/regular/F1_surprise_regular_51.wav  
  inflating: jvnv_v1/F1/surprise/regular/F1_surprise_regular_50.wav 

In [3]:
import glob
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from pickle import FRAME

In [4]:
# ------------------------------------------------------------
# 音声読み込み
# ------------------------------------------------------------
# sr=None とすることで、ファイルが元々持っているサンプリング周波数をそのまま使う。
# ここでは感情音声コーパス jvnv_v1 の「悲しみ / 通常発話」M2話者の 43 番目のファイルを読み込んでいる。
yA, sr = librosa.load("jvnv_v1/M2/sad/regular/M2_sad_regular_43.wav", sr=None)

# ------------------------------------------------------------
# VAD（Voice Activity Detection: 音声区間検出）の閾値・フレーム条件
# ------------------------------------------------------------
# VAD_THRESHOLD_RMS:
#   1フレームの実効値（RMS: root mean square）がこの値を超えたら
#   「有声（音がある）」と判定するための閾値。
#   単位は波形振幅のスケール（librosa のデフォルト：-1〜1）に依存する。
VAD_THRESHOLD_RMS = 0.01

# VAD_THRESHOLD_NORM_RMS:
#   正規化された RMS に対する閾値候補（例示としてコメントアウトされている）。
#   フレームごとの RMS を何らかの基準で正規化してから閾値判定したい場合に使える。
# VAD_THRESHOLD_NORM_RMS = 0.05

# FRAME_LENGTH:
#   1フレームの長さを「0.1 秒 × サンプリング周波数」で定義している。
#   例: sr=16000 の場合 → 1600 サンプル ≒ 100 ms。
FRAME_LENGTH = int(0.1 * sr)

# FRAME_SHIFT:
#   連続するフレームの開始位置のシフト量（サンプル数）。
#   ここでは固定値 256 サンプル（sr=16kHz なら約 16ms）で、フレーム長よりも短いためオーバーラップ分析になる。
FRAME_SHIFT = 256

# FRAME_SHIFT_SEC:
#   シフト量を秒単位で表したもの。時間軸にプロットするときなどに便利。
FRAME_SHIFT_SEC = FRAME_SHIFT / sr

# SILENCE_THRESHOLD_FRAMES:
#   「何フレーム連続で無音が続いたら本当に無音区間とみなすか」の閾値例。
#   ここでは 10 フレーム（≈ 10×16ms ≒ 160ms）分の無音が続いたら
#   1 つの無音区間として扱う、というような使い方を想定している。
SILENCE_THRESHOLD_FRAMES = 10


# ------------------------------------------------------------
# VAD のメイン処理
# ------------------------------------------------------------
# 与えられた波形 y をフレームごとに分割し、各フレームの RMS を計算して
# 閾値との比較で「有声(True) / 無声(False)」のフラグ列を返す。
def calculate_vad(y, threshold, frame_length, frame_shift):
    # 全体のサンプル長から、何フレーム取れるかを計算
    # （端数は切り捨て。端の余りサンプルは無視する簡易実装）
    frame_count = int(len(y) / frame_shift)

    # フレームごとの VAD 判定結果(True/False)を入れるリスト
    vad = []

    # 各フレームごとに RMS を計算して、閾値と比較する
    for f in range(frame_count):
        # f 番目のフレームの開始〜終了インデックスを算出
        start = f * frame_shift
        end = f * frame_shift + frame_length

        # 波形がフレーム長よりも短くなっている場合は、そのまま切り出す
        # （end が len(y) を超えるときは、NumPy のスライス仕様により自動的に末尾で切れる）
        frame = y[start:end]

        # フレーム内 RMS を計算
        #   rms = sqrt( (1/N) * Σ frame[n]^2 )
        # となっており、振幅の「平均的な大きさ」を表す指標になっている。
        if len(frame) > 0:
            rms = np.sqrt(np.mean(frame**2))
        else:
            # フレーム長が 0（ありえないはずだが安全のため）の場合は 0 とみなす
            rms = 0.0

        # 閾値より大きければ「音声あり」と判定
        # → True: 有声 / False: 無声
        vad.append(True if rms > threshold else False)

    # 各フレームごとの VAD 判定結果（長さ frame_count の True/False リスト）を返す
    return vad


# ------------------------------------------------------------
# 実データに対する VAD の実行
# ------------------------------------------------------------
# 上で定義した calculate_vad を用いて、読み込んだ音声 yA に対して
# 「フレームごとの有声/無声フラグ列」を計算する。
# ここで得られる vad は、例えば
# - 有声区間の時間範囲を抽出する
# - スペクトログラムや F0 を「有声部分だけ」プロットする
# といった用途にそのまま使える。
vad = calculate_vad(yA, VAD_THRESHOLD_RMS, FRAME_LENGTH, FRAME_SHIFT)